# Heat Map Interactivity 

In [1]:
from dash import Dash, html, dcc, Input, Output
import altair as alt
import dash_bootstrap_components as dbc
from vega_datasets import data
import panel as pn
from panel.interact import interact
import pandas as pd
import numpy as np
import sheworewhat as sww

# Basic interactivity

In [ ]:
df = data.cars()
countries = df["Origin"].unique().tolist()
countries

In [ ]:
# df = data.cars()
# countries = df["Origin"].unique().tolist()

# def plot_altair(zcol):
#     chart = alt.Chart(df).mark_circle().encode(
#         x='Year',
#         y="Miles_per_Gallon",
#         color=zcol).interactive()

#     return chart.to_html()

# app = Dash(__name__)
# server = app.server

# app.layout = html.Div([
#         dcc.Dropdown(
#             id='zcol', value='Origin',
#             options=[{'label': i, 'value': i} for i in df.columns]),
#         html.Iframe(
#             id='scatter',
#             style={'border-width': '0', 'width': '100%', 'height': '400px'},
#             srcDoc=plot_altair(zcol="Origin"))
# ])

# @app.callback(
#     Output('scatter', 'srcDoc'),
#     Input('zcol', 'value'))

# def update_output(zcol):
#     return plot_altair(zcol)

# if __name__ == '__main__':
#     app.run_server(
#         port = 8075,
#         debug=False)

# Trying it out with heatmap

In [ ]:
df = sww.closet_df()
worn_df = sww.worn(df)
top_10, df = sww.top_10_df()

In [ ]:
# sww.plot_heatmap(top_10, df, 0)

# Making button

In [ ]:
top_dict

In [ ]:
from dash import Dash, html, dcc, Input, Output
import altair as alt
import dash_bootstrap_components as dbc
import pandas as pd
import numpy as np

import sheworewhat as sww

top_dict, top_id, top_item, df = sww.top_10_df()

# need to make top 10 list a dictionary
app = Dash(__name__)
server = app.server

app.layout = html.Div([
        dcc.Dropdown(
            id='zcol',
            value=top_item[0],
            options=[{'label': item, 'value': i} for i, item in enumerate(top_item)])
])

if __name__ == '__main__':
    app.run_server(
        port = 8070,
        debug=False)

## Making button + graph
- this works but the two are not linked!
- omg this worked!

In [ ]:
from dash import Dash, html, dcc, Input, Output
import altair as alt
import dash_bootstrap_components as dbc
import pandas as pd
import numpy as np

import sheworewhat as sww

top_dict, top_id, top_item, df = sww.top_10_df()

def plot_heatmap(top_10, df, z=0):
    # column of day of week for one calender year
    time_df = pd.DataFrame()
    time_df["Date"] = pd.date_range(df["Date"].min(), periods=365)
    time_df["Day"] = time_df["Date"].dt.day_name()

    heatmap_data = df.loc[df["ID"] == top_10[z]]  # need to make this dynamic in plot
    item_name = heatmap_data["Brand"].iloc[0] + " " + heatmap_data["Item"].iloc[0]

    # isolate item data
    year = pd.merge(time_df, heatmap_data, how="outer", on="Date")
    year["Item"] = year["Item"].replace(np.nan, 0)
    year["Bool"] = np.where(year["Item"] == 0, 0, 1)

    # this is horrible to read lol
    # wrangling for prettier plotting
    week = time_df["Date"].dt.isocalendar()
    year["Week"] = week["week"].fillna(52)
    year["Week"] = year["Week"].fillna(52)
    year["First_day"] = year["Date"] - year["Date"].dt.weekday * np.timedelta64(1, "D")
    week = time_df["Date"].dt.strftime("%m-%d-%y")
    year["Week"] = year["First_day"].dt.strftime("%m-%d")
    year = year[["Date", "Day", "Item", "ID", "Bool", "Week"]]

    weekdays = [
        "Sunday",
        "Monday",
        "Tuesday",
        "Wednesday",
        "Thursday",
        "Friday",
        "Sunday",
    ]

    heat_plot = (
        alt.Chart(year, title=f"{item_name} in 2023")
        .mark_rect(
            stroke="white",
            strokeWidth=3,
            opacity=0.85,
        )
        .encode(
            alt.X(
                "Week:O",
                axis=alt.Axis(
                    labelAngle=-60,
                ),
            ),
            alt.Y("Day", sort=weekdays, title=""),
            alt.Color(
                "Bool",
                scale=alt.Scale(domain=[0, 1], range=["#e0ddd5", "#74a675"]),
                legend=None,
            ),
            alt.Tooltip(["Date", "Day"]),
        )
        .properties(height=200, width=600)
        .configure_axis(
            grid=False, domain=False, labelColor="#706f6c", titleColor="#706f6c"
        )
        .configure_title(color="#706f6c")
    )
    return heat_plot.to_html()

# need to make top 10 list a dictionary
app = Dash(__name__)
server = app.server

app.layout = html.Div([
        dcc.Dropdown(
            id='select-item',
            options=[{'label': item, 'value': i} for i, item in enumerate(top_item)]), # can switch to k,v instead of enumerate
        html.Iframe(
            id='heatmap_item',
            style={'border-width': '0', 'width': '100%', 'height': '400px'},
            srcDoc=plot_heatmap(top_id, df))
])

@app.callback(
    Output('heatmap_item', 'srcDoc'),
    Input('select-item', 'value'))

def update_output(z):
    return plot_heatmap(top_id, df, z)

if __name__ == '__main__':
    app.run_server(
        port = 8077,
        debug=False)

But can i get it to work without directly defining function in the script?
- probs need to update function calls
    - to html and sww.
    
... and it works! ok now to plug into the larger script

In [7]:
from dash import Dash, html, dcc, Input, Output
import altair as alt
import dash_bootstrap_components as dbc
import pandas as pd
import numpy as np

import sheworewhat as sww

top_id, top_item, df = sww.top_10_df()

# need to make top 10 list a dictionary
app = Dash(__name__)
server = app.server

app.layout = html.Div([
        dcc.Dropdown(
            id='select-item',
            options=[{'label': item, 'value': i} for i, item in enumerate(top_item)]), 
        html.Iframe(
            id='heatmap_item',
            style={'border-width': '0', 'width': '100%', 'height': '400px'},
            srcDoc=sww.plot_heatmap(top_id, df).to_html())
])

@app.callback(
    Output('heatmap_item', 'srcDoc'),
    Input('select-item', 'value'))

def update_output(z):
    return sww.plot_heatmap(top_id, df, z).to_html()

if __name__ == '__main__':
    app.run_server(
        port = 8073,
        debug=False)

Dash is running on http://127.0.0.1:8073/

Dash is running on http://127.0.0.1:8073/

Dash is running on http://127.0.0.1:8073/

Dash is running on http://127.0.0.1:8073/

 * Serving Flask app '__main__'
 * Debug mode: off


Address already in use
Port 8073 is in use by another program. Either identify and stop that program, or start the server with a different port.
ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/Users/jasmineortega/opt/miniconda3/lib/python3.9/site-packages/werkzeug/serving.py", line 911, in prepare_socket
    s.bind(server_address)
OSError: [Errno 48] Address already in use

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/jasmineortega/opt/miniconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py", line 3444, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/var/folders/qh/6chp8sds4c7gybyg8m5n7k3m0000gn/T/ipykernel_68249/2707076749.py", line 33, in <module>
    app.run_server(
  File "/Users/jasmineortega/opt/miniconda3/lib/python3.9/site-packages/dash/dash.py", line 2133, in run_server
    self.run(*args, **kwargs)
  File "/Users/jasmineortega/opt/miniconda3/lib/python3.9/site-packages/dash/dash.py", line 1980, in run
    self.server.run(host=host, port=port, debug=debug, **flask_run_options)
  File "/Users/jasmineortega/op

TypeError: object of type 'NoneType' has no len()

# Top 10 barchart + button

In [ ]:
closet = sww.closet_df()
worn_df = sww.worn(closet)

In [ ]:
def plot_mostworn(worn_df, item_name="Adidas Tennis Shoe"):
    most_worn = worn_df.nlargest(10, columns="Count")
    closet_comp = (
        alt.Chart(most_worn, title="Ten Most Worn Pieces in 2023")
        .mark_bar(
            cornerRadiusBottomRight=10,
            cornerRadiusTopRight=10,
            opacity=0.85,
        )
        .encode(
            alt.Y("Name", title="", axis=alt.Axis(labelAngle=-0), sort="-x"),
            alt.X("Count", title="Times Worn", axis=alt.Axis(tickMinStep=1)),
            alt.Tooltip("Count"),
            color=alt.condition(
                alt.datum.Name == item_name,  
                alt.value('#8ba88a'),     # highlights selected bar
                alt.value('#e0ddd5')   
    )
        )
        .configure_title(color="#706f6c")
        .configure_axis(
            labelColor="#706f6c", titleColor="#706f6c", grid=False, domain=False
        )
    )

    return closet_comp

In [ ]:
plot_mostworn(worn_df, "Zara Coat")

the code below works but then the dropdown menu is by ID wahh

In [ ]:
from dash import Dash, html, dcc, Input, Output
import altair as alt
import dash_bootstrap_components as dbc
import pandas as pd
import numpy as np

import sheworewhat as sww

closet = sww.closet_df()
worn_df = sww.worn(closet)
top_id, top_item, df = sww.top_10_df()

# need to make top 10 list a dictionary
app = Dash(__name__)
server = app.server

app.layout = html.Div([
        dcc.Dropdown(
            id='item_name',
            value=top_item[0],
            options=[{'value': item, 'label': i} for i, item in enumerate(top_item)]
        ), 
        html.Iframe(
            id='top10',
            style={'border-width': '0', 'width': '100%', 'height': '400px'},
            srcDoc=plot_mostworn(worn_df).to_html())
])

@app.callback(
    Output('top10', 'srcDoc'),
    Input('item_name', 'value'))

def update_output(item_name):
    return plot_mostworn(worn_df, item_name).to_html()


if __name__ == '__main__':
    app.run_server(
        port = 8075,
        debug=False)

Was able to workaround a single value being the output! (as list)

In [ ]:
def plot_mostworn(worn_df, item_name="Adidas Tennis Shoe"):
    most_worn = worn_df.nlargest(10, columns="Count")
    closet_comp = (
        alt.Chart(most_worn, title="Ten Most Worn Pieces in 2023")
        .mark_bar(
            cornerRadiusBottomRight=10,
            cornerRadiusTopRight=10,
            opacity=0.85,
        )
        .encode(
            alt.Y("Name", title="", axis=alt.Axis(labelAngle=-0), sort="-x"),
            alt.X("Count", title="Times Worn", axis=alt.Axis(tickMinStep=1)),
            alt.Tooltip("Count"),
            color=alt.condition(
                alt.datum.Name == item_name,  
                alt.value('#8ba88a'),     # highlights selected bar
                alt.value('#e0ddd5')   
    )
        )
        .configure_title(color="#706f6c")
        .configure_axis(
            labelColor="#706f6c", titleColor="#706f6c", grid=False, domain=False
        )
    )

    return closet_comp

In [ ]:
from dash import Dash, html, dcc, Input, Output
import altair as alt
import dash_bootstrap_components as dbc
import pandas as pd
import numpy as np

import sheworewhat as sww

closet = sww.closet_df()
worn_df = sww.worn(closet)
top_id, top_item, df = sww.top_10_df()

# need to make top 10 list a dictionary
app = Dash(__name__)
server = app.server

app.layout = html.Div([
        dcc.Dropdown(
            id='item_name',
            value=top_item[0], # this isn't displaying for some reason
            options=[{'label': item, 'value': [i, item]} for i, item in enumerate(top_item)]
        ), 
        html.Iframe(
            id='top10',
            style={'border-width': '0', 'width': '100%', 'height': '400px'},
            srcDoc=plot_mostworn(worn_df, top_item[0]).to_html())
])

@app.callback(
    Output('top10', 'srcDoc'),
    Input('item_name', 'value'))

def update_output(item_name):
    x = item_name[1]
    return plot_mostworn(worn_df, x).to_html()


if __name__ == '__main__':
    app.run_server(
        port = 8077,
        debug=False)

# To finally link the button and graph

In [2]:
closet = sww.closet_df()
worn_df = sww.worn(closet)

In [3]:
def plot_mostworn(worn_df, item_name="Adidas Tennis Shoe"):
    most_worn = worn_df.nlargest(10, columns="Count")
    closet_comp = (
        alt.Chart(most_worn, title="Ten Most Worn Pieces in 2023")
        .mark_bar(
            cornerRadiusBottomRight=10,
            cornerRadiusTopRight=10,
            opacity=0.85,
        )
        .encode(
            alt.Y("Name", title="", axis=alt.Axis(labelAngle=-0), sort="-x"),
            alt.X("Count", title="Times Worn", axis=alt.Axis(tickMinStep=1)),
            alt.Tooltip("Count"),
            color=alt.condition(
                alt.datum.Name == item_name,  
                alt.value('#8ba88a'),     # highlights selected bar
                alt.value('#e0ddd5')   
    )
        )
        .configure_title(color="#706f6c")
        .configure_axis(
            labelColor="#706f6c", titleColor="#706f6c", grid=False, domain=False
        )
    )

    return closet_comp

In [11]:
from dash import Dash, html, dcc, Input, Output
import altair as alt
import dash_bootstrap_components as dbc
import pandas as pd
import numpy as np

import sheworewhat as sww

closet = sww.closet_df()
worn_df = sww.worn(closet)
top_id, top_item, df = sww.top_10_df()

# need to make top 10 list a dictionary
app = Dash(__name__)
server = app.server

app.layout = html.Div([
        dcc.Dropdown(
            id='item_name',
            value=top_item[0], # this isn't displaying for some reason
            options=[{'label': item, 'value': [i, item]} for i, item in enumerate(top_item)]
        ), 
        html.Iframe(
            id='top10',
            style={'border-width': '0', 'width': '100%', 'height': '400px'},
            srcDoc=plot_mostworn(worn_df, top_item[0]).to_html()
        ), 
        html.Iframe(
            id='heatmap_item',
            style={'border-width': '0', 'width': '100%', 'height': '400px'},
            srcDoc=sww.plot_heatmap(top_id, df).to_html())
        
])

@app.callback(
    Output('top10', 'srcDoc'),
    Input('item_name', 'value'))

def update_highlight(item_name):
    x = item_name[1]
    return plot_mostworn(worn_df, x).to_html()

@app.callback(
    Output('heatmap_item', 'srcDoc'),
    Input('item_name', 'value'))

def update_output(item_name):
    y = item_name[0]
    return sww.plot_heatmap(top_id, df, y).to_html()


if __name__ == '__main__':
    app.run_server(
        port = 8078,
        debug=False)

Dash is running on http://127.0.0.1:8078/

Dash is running on http://127.0.0.1:8078/

Dash is running on http://127.0.0.1:8078/

Dash is running on http://127.0.0.1:8078/

Dash is running on http://127.0.0.1:8078/

Dash is running on http://127.0.0.1:8078/

Dash is running on http://127.0.0.1:8078/

Dash is running on http://127.0.0.1:8078/



INFO:__main__:Dash is running on http://127.0.0.1:8078/



 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:8078
INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:8078
Press CTRL+C to quit
INFO:werkzeug:Press CTRL+C to quit
127.0.0.1 - - [17/Jan/2023 21:57:49] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [17/Jan/2023 21:57:49] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [17/Jan/2023 21:57:49] "GET /_dash-layout HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [17/Jan/2023 21:57:49] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [17/Jan/2023 21:57:49] "GET /_dash-dependencies HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [17/Jan/2023 21:57:49] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [17/Jan/2023 21:57:49] "GET /_dash-component-suites/dash/dcc/async-dropdown.js HTTP/1.1

Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "/Users/jasmineortega/opt/miniconda3/lib/python3.9/site-packages/flask/app.py", line 2525, in wsgi_app
    response = self.full_dispatch_request()
  File "/Users/jasmineortega/opt/miniconda3/lib/python3.9/site-packages/flask/app.py", line 1822, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/Users/jasmineortega/opt/miniconda3/lib/python3.9/site-packages/flask/app.py", line 1820, in full_dispatch_request
    rv = self.dispatch_request()
  File "/Users/jasmineortega/opt/miniconda3/lib/python3.9/site-packages/flask/app.py", line 1796, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**view_args)
  File "/Users/jasmineortega/opt/miniconda3/lib/python3.9/site-packages/dash/dash.py", line 1274, in dispatch
    ctx.run(
  File "/Users/jasmineortega/opt/miniconda3/lib/python3.9/site-packages/dash/_callback.py", line 440, in add_context
    output_val

...

INFO:werkzeug:127.0.0.1 - - [17/Jan/2023 21:57:49] "GET /_dash-component-suites/dash/dcc/async-dropdown.js HTTP/1.1" 304 -
127.0.0.1 - - [17/Jan/2023 21:57:49] "POST /_dash-update-component HTTP/1.1" 200 -


Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "/Users/jasmineortega/opt/miniconda3/lib/python3.9/site-packages/flask/app.py", line 2525, in wsgi_app
    response = self.full_dispatch_request()
  File "/Users/jasmineortega/opt/miniconda3/lib/python3.9/site-packages/flask/app.py", line 1822, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/Users/jasmineortega/opt/miniconda3/lib/python3.9/site-packages/flask/app.py", line 1820, in full_dispatch_request
    rv = self.dispatch_request()
  File "/Users/jasmineortega/opt/miniconda3/lib/python3.9/site-packages/flask/app.py", line 1796, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**view_args)
  File "/Users/jasmineortega/opt/miniconda3/lib/python3.9/site-packages/dash/dash.py", line 1274, in dispatch
    ctx.run(
  File "/Users/jasmineortega/opt/miniconda3/lib/python3.9/site-packages/dash/_callback.py", line 440, in add_context
    output_val

INFO:werkzeug:127.0.0.1 - - [17/Jan/2023 21:57:49] "POST /_dash-update-component HTTP/1.1" 200 -


Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "/Users/jasmineortega/opt/miniconda3/lib/python3.9/site-packages/flask/app.py", line 2525, in wsgi_app
    response = self.full_dispatch_request()
  File "/Users/jasmineortega/opt/miniconda3/lib/python3.9/site-packages/flask/app.py", line 1822, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/Users/jasmineortega/opt/miniconda3/lib/python3.9/site-packages/flask/app.py", line 1820, in full_dispatch_request
    rv = self.dispatch_request()
  File "/Users/jasmineortega/opt/miniconda3/lib/python3.9/site-packages/flask/app.py", line 1796, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**view_args)
  File "/Users/jasmineortega/opt/miniconda3/lib/python3.9/site-packages/dash/dash.py", line 1274, in dispatch
    ctx.run(
  File "/Users/jasmineortega/opt/miniconda3/lib/python3.9/site-packages/dash/_callback.py", line 440, in add_context
    output_val

ERROR:__main__:Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "/Users/jasmineortega/opt/miniconda3/lib/python3.9/site-packages/flask/app.py", line 2525, in wsgi_app
    response = self.full_dispatch_request()
  File "/Users/jasmineortega/opt/miniconda3/lib/python3.9/site-packages/flask/app.py", line 1822, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/Users/jasmineortega/opt/miniconda3/lib/python3.9/site-packages/flask/app.py", line 1820, in full_dispatch_request
    rv = self.dispatch_request()
  File "/Users/jasmineortega/opt/miniconda3/lib/python3.9/site-packages/flask/app.py", line 1796, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**view_args)
  File "/Users/jasmineortega/opt/miniconda3/lib/python3.9/site-packages/dash/dash.py", line 1274, in dispatch
    ctx.run(
  File "/Users/jasmineortega/opt/miniconda3/lib/python3.9/site-packages/dash/_callback.py", line 440, in add_context